In [21]:
# import libraries
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [22]:
#  load train data
data = np.genfromtxt('Downloads/0000000000002419_training_ccpp_x_y_train.csv', delimiter=',')
X_train = data[:, 0:4]
Y_train = data[:,4]

In [23]:
# load test data
X_test = np.genfromtxt('Downloads/0000000000002419_test_ccpp_x_test.csv', delimiter=',')

In [25]:
# for feature scaling
temp = np.vstack((X_train ,X_test))
temp.shape

(9568, 4)

In [26]:
# feature scaling
temp = preprocessing.scale(temp)

In [29]:
temp

array([[-1.48565576, -1.25328692,  1.30857232,  0.7576299 ],
       [ 0.28700279,  0.30645519,  0.66530935, -0.44994522],
       [-0.40407908, -0.42383778, -0.28779757,  0.37337032],
       ...,
       [ 0.62650514,  0.93995287, -0.70204545,  1.27271583],
       [ 0.5151269 , -0.90545341,  0.11803064, -0.54789374],
       [ 0.28297707,  0.4693546 ,  0.95999788,  0.80489178]])

In [30]:
df = pd.DataFrame(temp)
df.head()

,0,1,2,3
0,-1.485656,-1.253287,1.308572,0.757630
1,0.287003,0.306455,0.665309,-0.449945
2,-0.404079,-0.423838,-0.287798,0.373370
3,1.573891,1.338938,-0.688574,-0.882837
4,-1.402458,-1.123440,0.638366,1.311758


In [32]:
# splt data
X_train = df.iloc[0:7176,:].values
X_test = df.iloc[7176:,:].values

In [36]:
# data = np.genfromtxt('Downloads/0000000000002419_training_ccpp_x_y_train.csv', delimiter=',')
# X_train = data[:, 0:4]
# Y_train = data[:,4]

columns = X_train.shape[1]

# self pair
for i in range(columns):
    new_column = (X_train[:, i] * X_train[:, i]).reshape(-1,1)
    X_train = np.append(X_train, new_column, axis=1)
    

ones = np.ones(X_train.shape[0]).reshape(-1, 1)
X_train = np.append(X_train, ones, axis=1)

# X_test = np.genfromtxt('Downloads/0000000000002419_test_ccpp_x_test.csv', delimiter=',')


columns = X_test.shape[1]

# self pair
for i in range(columns):
    new_column = (X_test[:, i] * X_test[:, i]).reshape(-1,1)
    X_test = np.append(X_test, new_column, axis=1)

test_ones = np.ones(X_test.shape[0]).reshape(-1, 1)
X_test = np.append(X_test, test_ones, axis=1)


data.shape, X_train.shape, Y_train.shape, X_test.shape

((7176, 5), (7176, 9), (7176,), (2392, 9))

In [37]:
df = pd.DataFrame(X_train)
df.head()

,0,1,2,3,4,5,6,7,8
0,-1.485656,-1.253287,1.308572,0.757630,2.207173,1.570728,1.712362,0.574003,1.0
1,0.287003,0.306455,0.665309,-0.449945,0.082371,0.093915,0.442637,0.202451,1.0
2,-0.404079,-0.423838,-0.287798,0.373370,0.163280,0.179638,0.082827,0.139405,1.0
3,1.573891,1.338938,-0.688574,-0.882837,2.477133,1.792756,0.474134,0.779400,1.0
4,-1.402458,-1.123440,0.638366,1.311758,1.966887,1.262116,0.407512,1.720710,1.0


In [38]:
def step_gradient(X_train, Y_train, learning_rate, m,j):
    m_j = 0
    n_data_pts = X_train.shape[0]
    N = len(m)
    for i in range(n_data_pts):
        x_i = X_train[i, :]
        y_i = Y_train[i]
        temp_sum = 0
        for k in range(N):
            temp_sum += m[k]*x_i[k]
        temp_sum = y_i - temp_sum
        ## complete formula
        m_j += (-2/n_data_pts) * (temp_sum) * x_i[j]
    # update m[j] and return
    m[j] = m[j] - (learning_rate*m_j)
    return m[j]

In [39]:

def gradient_descent(X_train, Y_train, learning_rate, num_iterations):
    m = [0]*(X_train.shape[1])
    m[-1] = 1 #c
    N = len(m)
    for i in range(num_iterations):
        for j in range(N):
            m[j] = step_gradient(X_train, Y_train, learning_rate, m,j)
        a = cost(X_train, Y_train, m)
        if (i%10==0) or (i ==(num_iterations-1)):
            print("iteration",i," cost:",a)
    return m

In [40]:
def cost(X_train, Y_train, m):
    cost = 0
    n_data_pts = len(X_train)
    N = len(m)
    for i in range(n_data_pts):
        x_i = X_train[i, :]
        y_i = Y_train[i]
        temp_sum = 0
        for k in range(N):
            temp_sum += m[k]*x_i[k]
        temp_sum = y_i - temp_sum
        cost += (1/n_data_pts) * ((temp_sum)**2)
    return cost

In [41]:
def predict(X_test, m):
    n_fts = X_train.shape[1]
    n_m = np.array(m).reshape(n_fts, 1)
    return np.dot(X_test, n_m)

In [42]:
df_test = pd.DataFrame(X_test)

In [48]:
def run():
    num_iterations = 1200
    learning_rate = 0.1
    m = gradient_descent(X_train, Y_train, learning_rate, num_iterations)
    print(m)
    Y_pred = predict(X_test, m)
    np.savetxt('ccpp_prediction_1.csv', Y_pred, fmt='%.5f', delimiter=',')

In [49]:
run()

iteration 0  cost: 52315.54312709577
iteration 10  cost: 14576.587286369419
iteration 20  cost: 5557.305749012124
iteration 30  cost: 2352.8383499761667
iteration 40  cost: 1070.5671542507635
iteration 50  cost: 526.0764357226037
iteration 60  cost: 282.4889985999844
iteration 70  cost: 166.57426240175104
iteration 80  cost: 107.24967780160212
iteration 90  cost: 74.4833146784253
iteration 100  cost: 55.09299258334312
iteration 110  cost: 42.975631263683304
iteration 120  cost: 35.1057480674488
iteration 130  cost: 29.86426919037121
iteration 140  cost: 26.31859187769264
iteration 150  cost: 23.897630025614376
iteration 160  cost: 22.23557388939203
iteration 170  cost: 21.090920067813666
iteration 180  cost: 20.301169958296846
iteration 190  cost: 19.75571999576615
iteration 200  cost: 19.378776577609813
iteration 210  cost: 19.118195575123313
iteration 220  cost: 18.93802179150347
iteration 230  cost: 18.813430739823136
iteration 240  cost: 18.727270247477147
iteration 250  cost: 18.6

In [10]:
a=np.array([1,2,3])
a

array([1, 2, 3])

In [11]:
a = np.array([1,2])
a

array([1, 2])